# AUTOMATIC1111's Stable Diffusion WebUI

https://github.com/AUTOMATIC1111/stable-diffusion-webui

Loosely based on https://colab.research.google.com/drive/1kw3egmSn-KgWsikYvOMjJkVDsPLjEMzl

**Guides**
- [Getting started on Paperspace](https://github.com/Engineer-of-Stuff/stable-diffusion-paperspace/blob/main/Docs/Paperspace%20Guide%20for%20Retards.md)
- [Using the WebUI](https://rentry.org/voldy)
- [Using the Inpainter](https://rentry.org/drfar)
- [Textual Inversion](https://rentry.org/aikgx)
- [Crowd-Sourced Prompts](https://lexica.art/)
- [Artist Name Prompts](https://sgreens.notion.site/sgreens/4ca6f4e229e24da6845b6d49e6b08ae7?v=fdf861d1c65d456e98904fe3f3670bd3)
- [Stable Diffusion Models](https://cyberes.github.io/stable-diffusion-models)
- [Textual Inversion Models](https://cyberes.github.io/stable-diffusion-textual-inversion-models/)
- [Have I Been Trained?](https://haveibeentrained.com/)

## Installation and Setup

You must reinstall everything each time you restart the machine. If already downloaded, dependencies will be auto-updated.

**Where to store the models**

`/storage/` is persistent storage shared across all machines on your account.

`/notebooks/` is storage for this notebook only.

`/tmp/` <mark style="background-color:lime">is not a persistent directory, meaning your files there will be deleted when the machine turns off.</mark>

<br>

<mark style="background-color: #ff780082">If you are having storage issues</mark>, set `repo_storage_dir` to `/tmp/stable-diffusion`.

<br>

<mark>You must uncomment the correct section and run the block below or else the notebook won't work!</mark>

In [2]:
# Free tier
# free_tier = True                                     # Enables the creation of symlinks back to /notebooks/
# model_storage_dir = '/tmp/stable-diffusion/models'   # Where the models will be downloaded to.
# repo_storage_dir = '/notebooks'                      # Where the repository will be downloaded to.
# activate_xformers = False                            # Have you installed and built Xformers? This will enable it in the WebUI. There's a block in 
                                                       # the tools section that will guide you through building it.
# Paid Tier
free_tier = False
model_storage_dir = '/storage/models'
repo_storage_dir = '/notebooks'
activate_xformers = True

link_novelai_anime_vae = True                          # Enables the linking of animevae.pt to each of the NovelAI models.
                                                       # Set to True if you've downloaded both the NovelAI models and hypernetworks.
# Don't put a trailing slash on directory paths.
# To reset your storage directory, rerun this cell.

# ===============================================================
# Save variables to Jupiter's temp storage so we can access it even if the kernel restarts.
%store free_tier model_storage_dir repo_storage_dir activate_xformers link_novelai_anime_vae

Stored 'free_tier' (bool)
Stored 'model_storage_dir' (str)
Stored 'repo_storage_dir' (str)
Stored 'activate_xformers' (bool)
Stored 'link_novelai_anime_vae' (bool)


**Don't forget, there's a script to update this notebook to [the latest version](https://github.com/Engineer-of-Stuff/stable-diffusion-paperspace/blob/main/StableDiffusionUI_Voldemort_paperspace.ipynb) on GitHub.**

### Clone the central repository

In [3]:
import os
%store -r free_tier model_storage_dir repo_storage_dir activate_xformers link_novelai_anime_vae
%cd /notebooks/

def delete_broken_symlinks(path):
    # make sure to pass this function a path without a trailing slash
    for file in os.listdir(path):
        if os.path.islink(f'{path}/{file}') and not os.path.exists(os.readlink(f'{path}/{file}')):
            print(f'Symlink broken, removing: {file}')
            os.unlink(f'{path}/{file}')

def update_repo_if_not_exists(path, repo_clone_url, pre=None):
    if pre is not None:
        pre()    
    if not os.path.exists(path):
        !git clone "{repo_clone_url}" "{path}"
    else:
        print(f'{repo_clone_url.split("/")[-1]} already downloaded, updating...')
        !cd "{path}" && git pull # no % so we don't interfere with the main process

def init_free():
    if (free_tier and repo_storage_dir != '/notebooks'):
        delete_broken_symlinks('/notebooks/') # remove broken symlinks since it might have been installed in a non-persistent directory
        if not os.path.exists(repo_storage_dir):
            !mkdir -p "{repo_storage_dir}"
            !ln -s "{repo_storage_dir}" /notebooks/
            !ls -la /notebooks/stable-diffusion
update_repo_if_not_exists(f'{repo_storage_dir}/stable-diffusion-webui', 'https://github.com/AUTOMATIC1111/stable-diffusion-webui', init_free)

/notebooks
stable-diffusion-webui already downloaded, updating...
Already up to date.


### Install requirements and download repositories

In [4]:
%store -r free_tier model_storage_dir repo_storage_dir activate_xformers link_novelai_anime_vae
%cd "{repo_storage_dir}/stable-diffusion-webui"

# Import launch.py which will automatically run the install script but not launch the WebUI.
# They require a few specific external git repo commits so we have to do it their way. 
# !python3 launch.py --exit
import launch
launch.prepare_enviroment()

# Install requirements for this notebook
!pip install requests validators

# latent-diffusion is a requirement but launch.py isn't downloading it so we'll do it manually.
if not os.path.exists(f'{repo_storage_dir}/stable-diffusion-webui/repositories/latent-diffusion'):
    !git clone https://github.com/crowsonkb/k-diffusion.git "{repo_storage_dir}/stable-diffusion-webui/repositories/k-diffusion"
    !git clone https://github.com/Hafiidz/latent-diffusion.git "{repo_storage_dir}/stable-diffusion-webui/repositories/latent-diffusion"

# Download the GFPGAN face restorer.
if not os.path.exists(f'{repo_storage_dir}/stable-diffusion-webui/GFPGANv1.3.pth'):
    !wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -O "{repo_storage_dir}/stable-diffusion-webui/GFPGANv1.3.pth"
else:
    print('GFPGANv1.3.pth already downloaded')

# Download popular custom scripts. This is basically remote code execution so be careful.
# See https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Custom-Scripts
import os
import shutil
import requests
!pip install moviepy==1.0.3
!apt update
!apt install -y potrace python3-tk

def download_file_dir(url, output_dir):
    # output_dir must have a trailing slash
    local_filename = url.split('/')[-1]
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(f'{output_dir}{local_filename}', 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    return local_filename
def do_script_download(scripts_list, domain, path):
    for item in scripts_list:
        download_file_dir(f'https://{domain}/{item}', path)
        print(f'{item.split("/")[-1]} downloaded...')

do_script_download([
    'GRMrGecko/stable-diffusion-webui-automatic/advanced_matrix/scripts/advanced_prompt_matrix.py',
#    'jtkelm2/stable-diffusion-webui-1/master/scripts/wildcards.py',
    'adieyal/sd-dynamic-prompting/main/dynamic_prompting.py',
    'dfaker/stable-diffusion-webui-cv2-external-masking-script/main/external_masking.py',
    'memes-forever/Stable-diffusion-webui-video/main/videos.py',
    'yownas/seed_travel/main/scripts/seed_travel.py',
    'Animator-Anon/Animator/main/animation.py',
    'Filarius/stable-diffusion-webui/master/scripts/vid2vid.py',
    'GeorgLegato/Txt2Vectorgraphics/main/txt2vectorgfx.py',
    'yownas/shift-attention/main/scripts/shift_attention.py',
    'DiceOwl/StableDiffusionStuff/main/loopback_superimpose.py',
    'Engineer-of-Stuff/stable-diffusion-paperspace/main/lfs/save_steps.py',
    'Pfaeff/sd-web-ui-scripts/main/moisaic.py',
    'yownas/shift-attention/main/scripts/shift_attention.py',
    'feffy380/prompt-morph/master/prompt_morph.py'
], 'raw.githubusercontent.com', f'{repo_storage_dir}/stable-diffusion-webui/scripts/')

do_script_download([
    'dfaker/f88aa62e3a14b559fe4e5f6b345db664/raw/791dabfa0ab26399aa2635bcbc1cf6267aa4ffc2/alternate_sampler_noise_schedules.py',
    'camenduru/9ec5f8141db9902e375967e93250860f/raw/c1a03eb447548adbef1858c0e69d3567a390d2f4/run_n_times.py'
], 'gist.githubusercontent.com', f'{repo_storage_dir}/stable-diffusion-webui/scripts/')
    
# Download and set up txt2img2img
update_repo_if_not_exists(f'{repo_storage_dir}/stable-diffusion-webui/txt2img2img_root', 'https://github.com/ThereforeGames/txt2img2img.git')
!cp -r "{repo_storage_dir}/stable-diffusion-webui/txt2img2img_root/scripts" "{repo_storage_dir}/stable-diffusion-webui"
!cp -r "{repo_storage_dir}/stable-diffusion-webui/txt2img2img_root/txt2img2img" "{repo_storage_dir}/stable-diffusion-webui"
!cp -r "{repo_storage_dir}/stable-diffusion-webui/txt2img2img_root/venv" "{repo_storage_dir}/stable-diffusion-webui"
    
# Download and set up txt2mask
update_repo_if_not_exists(f'{repo_storage_dir}/stable-diffusion-webui/txt2mask', 'https://github.com/ThereforeGames/txt2mask.git')
!echo "Copying txt2mask..."
!cp -r "{repo_storage_dir}/stable-diffusion-webui/txt2mask/repositories/clipseg" "{repo_storage_dir}/stable-diffusion-webui/repositories"
!cp -r "{repo_storage_dir}/stable-diffusion-webui/txt2mask/scripts/" "{repo_storage_dir}/stable-diffusion-webui/"

# Do the wildcard script
!mkdir -p "{repo_storage_dir}/stable-diffusion-webui/scripts/wildcards"
do_script_download([
    'jtkelm2/stable-diffusion-webui-1/master/scripts/wildcards/adjective.txt',
    'jtkelm2/stable-diffusion-webui-1/master/scripts/wildcards/artist.txt',
    'jtkelm2/stable-diffusion-webui-1/master/scripts/wildcards/genre.txt',
    'jtkelm2/stable-diffusion-webui-1/master/scripts/wildcards/site.txt',
    'jtkelm2/stable-diffusion-webui-1/master/scripts/wildcards/style.txt'
], 'raw.githubusercontent.com', f'{repo_storage_dir}/stable-diffusion-webui/scripts/wildcards/')

if activate_xformers:
    import os
    import time
    from IPython.display import HTML
    from IPython.display import clear_output
    from subprocess import getoutput

    s = getoutput('nvidia-smi')
    if 'A4000' in s:
      gpu = 'A4000'
    elif 'A100' in s:
      gpu = 'A100'
    elif 'P5000' in s:
      gpu = 'P5000'
    elif 'A6000' in s:
      gpu = 'A6000'
    elif 'M4000' in s:
      gpu = 'M4000'
    elif 'RTX 5000' in s:
      gpu = 'RTX5000'

    while True:
        try: 
            gpu=='A4000'or gpu=='P5000'or gpu=='M4000'or gpu=='RTX5000'or gpu=='A6000'or gpu=='A100'
            break
        except:
            pass
        print('it seems that your GPU is not supported at the moment')
        time.sleep(5)

    if (gpu=='A4000'):
      %pip install https://github.com/Cyberes/xformers-compiled/raw/main/A4000/xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl
    if (gpu=='A100'):
      %pip install https://github.com/MarseyLivesMatter/xformers-compiled/raw/main/A100/xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl
    elif (gpu=='A6000'):
      %pip install https://github.com/MarseyLivesMatter/xformers-compiled/raw/main/A4000/xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl

    elif (gpu=='P5000'):
      %pip install https://github.com/MarseyLivesMatter/stable-diffusion-paperspace/raw/main/precompiled/P5000/xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl

    elif (gpu=='M4000'):
      %pip install https://github.com/MarseyLivesMatter/stable-diffusion-paperspace/raw/main/precompiled/M4000/xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl

    elif (gpu=='RTX5000'):
      %pip install https://github.com/MarseyLivesMatter/xformers-compiled/raw/main/RTX%205000/xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl

# Make sure your models storage directory exists
!mkdir -p "{model_storage_dir}/hypernetworks"
!mkdir -p "{repo_storage_dir}/stable-diffusion-webui/models/hypernetworks"

# Make sure the log dir exists

# Link the output folders to /notebooks/outputs
!mkdir -p "{repo_storage_dir}/stable-diffusion-webui/log/images"
!mkdir -p "{repo_storage_dir}/stable-diffusion-webui/outputs"
!ln -s "{repo_storage_dir}/stable-diffusion-webui/outputs" /notebooks/
!ln -s "{repo_storage_dir}/stable-diffusion-webui/log" "{repo_storage_dir}/stable-diffusion-webui/outputs"


!echo "Done!"

/notebooks/stable-diffusion-webui
Python 3.9.13 (main, May 23 2022, 22:01:06) 
[GCC 9.4.0]
Commit hash: 737eb28faca8be2bb996ee0930ec77d1f7ebd939
Installing requirements for Web UI
GFPGANv1.3.pth already downloaded
Hit:1 https://deb.nodesource.com/node_16.x focal InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease           
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease                         m
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]        
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1224 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [2737 kB]
Fetched 4297 kB in 1s (3187 kB/s)                    
Reading package lists... Done
Building dependency tree       
Reading state information... Done
98 packages can be

### Download the Model

I've provided a few different ways of aquiring the models. Try the torrent option first. You don't need to repeat this step if you've already downloaded the models.

There are a few additional models available here: https://cyberes.github.io/stable-diffusion-models

If you're interested in textual inversion, here's the database: https://cyberes.github.io/stable-diffusion-textual-inversion-models

**Filesize and Storage Disclaimer**

Paperspace free tier has only 5GB of storage space. If you're having storage issues, here's a few suggestions.
1. Download everything to `/tmp/`
2. Add a payment method to your account. Storage overages are billed at \$0.29/GB and billing occurs monthly and runs at midnight on the first of each month. With a payment method on file, Paperspace will let you use more storage and if you time it right you shouldn't actually be charged for it.
3. Upgrade to a Pro account. They'll give you 15GB and you'll get longer runtimes and more powerful free GPUs.
4. Use my referral code `KQLRH37` You'll get \$10 credit that you should be able to put towards the storage overage charges. Redeem the code at the bottom of the Billing page.

If you're on free tier, only download one model.

**Torrent Instructions**

Aria2 may show some errors/warnings while downloading. Those are fine, when it eventually says "Download Complete" that means everything worked as it should.

#### Standard Model

**Torrent**

In [ ]:
%store -r free_tier model_storage_dir repo_storage_dir activate_xformers link_novelai_anime_vae
!apt update
!apt install -y aria2
%cd $model_storage_dir
!aria2c --seed-time=0 --max-overall-upload-limit=1K --bt-max-peers=120 --summary-interval=0 --file-allocation=none "magnet:?xt=urn:btih:3A4A612D75ED088EA542ACAC52F9F45987488D1C&tr=udp://tracker.opentrackr.org:1337/announce"

#### Waifu Diffusion

The original Stable Diffusion anime finetune.

**v1.3**

In [ ]:
%store -r free_tier model_storage_dir repo_storage_dir activate_xformers link_novelai_anime_vae
!apt update
!apt install -y aria2
%cd "{model_storage_dir}"
!aria2c --seed-time=0 --bt-max-peers=120 --summary-interval=0 --file-allocation=none "magnet:?xt=urn:btih:AWJJJZNFOOK7R2XXXGZ4GFNKUEU2TSFP&dn=wd-v1-3-float16.ckpt&xl=2132889245&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce"

**v1.2**

In [ ]:
%store -r free_tier model_storage_dir repo_storage_dir activate_xformers link_novelai_anime_vae
!apt update
!apt install -y aria2
%cd "{model_storage_dir}"
!aria2c --seed-time=0 --max-overall-upload-limit=1K --bt-max-peers=120 --summary-interval=0 --file-allocation=none "magnet:?xt=urn:btih:153590FD7E93EE11D8DB951451056C362E3A9150&dn=wd-v1-2-full-ema-pruned.ckpt&tr=udp://tracker.opentrackr.org:1337"

#### trinart_stable_diffusion_v2

Another anime finetune. Designed to nudge SD to an anime/manga style. Seems to be more "stylized" and "artistic" than Waifu Diffusion, if that makes any sense.

The 60,000 steps version is the original, the 115,000 and 95,000 versions is the 60,000 with additional training. Use the 60,000 step version if the style nudging is too much.

[See the comparison here.](https://cyberes.github.io/stable-diffusion-models/#model-comparison)

**60000**

In [ ]:
%store -r free_tier model_storage_dir repo_storage_dir activate_xformers link_novelai_anime_vae
!wget https://huggingface.co/naclbit/trinart_stable_diffusion_v2/resolve/main/trinart2_step60000.ckpt -O "{model_storage_dir}/trinart2_step60000.ckpt"

**95000**

In [ ]:
%store -r free_tier model_storage_dir repo_storage_dir activate_xformers link_novelai_anime_vae
!wget https://huggingface.co/naclbit/trinart_stable_diffusion_v2/resolve/main/trinart2_step95000.ckpt -O "{model_storage_dir}/trinart2_step95000.ckpt"

**115000**

In [ ]:
%store -r free_tier model_storage_dir repo_storage_dir activate_xformers link_novelai_anime_vae
!wget https://huggingface.co/naclbit/trinart_stable_diffusion_v2/resolve/main/trinart2_step115000.ckpt -O "{model_storage_dir}/trinart2_step115000.ckpt"

#### NovelAI Leak

**animefull-final-pruned**

In [ ]:
%store -r free_tier model_storage_dir repo_storage_dir activate_xformers link_novelai_anime_vae
!apt update
!apt install -y aria2
metalink = 'magnet:?xt=urn:btih:5bde442da86265b670a3e5ea3163afad2c6f8ecc&dn=novelaileak&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2F9.rarbg.com%3A2810%2Fannounce&tr=udp%3A%2F%2Ftracker.openbittorrent.com%3A6969%2Fannounce&tr=http%3A%2F%2Ftracker.openbittorrent.com%3A80%2Fannounce&tr=udp%3A%2F%2Fopentracker.i2p.rocks%3A6969%2Fannounce'
import re
infohash = re.search("^magnet:\?xt=urn:btih:(.*?)&.*?$", metalink).group(1)
import subprocess
tmp_dir = subprocess.check_output(['mktemp', '-d']).decode('ascii').strip('\n')
%cd "{tmp_dir}"
# Have to convert the metalink to a torrent file so aria2c can read the files inside
!aria2c -d . --bt-metadata-only=true --bt-save-metadata=true --bt-max-peers=120 --summary-interval=0 --file-allocation=none "magnet:?xt=urn:btih:5bde442da86265b670a3e5ea3163afad2c6f8ecc&dn=novelaileak&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2F9.rarbg.com%3A2810%2Fannounce&tr=udp%3A%2F%2Ftracker.openbittorrent.com%3A6969%2Fannounce&tr=http%3A%2F%2Ftracker.openbittorrent.com%3A80%2Fannounce&tr=udp%3A%2F%2Fopentracker.i2p.rocks%3A6969%2Fannounce"
!aria2c --select-file=64,65 --seed-time=0 --max-overall-upload-limit=1K --bt-max-peers=120 --summary-interval=0 --file-allocation=none "{infohash}.torrent"
!mv "novelaileak/stableckpt/animefull-final-pruned/config.yaml" "{model_storage_dir}/novelai-animefull-final-pruned.yaml"
!mv "novelaileak/stableckpt/animefull-final-pruned/model.ckpt" "{model_storage_dir}/novelai-animefull-final-pruned.ckpt"

**animesfw-final-pruned**

In [ ]:
%store -r free_tier model_storage_dir repo_storage_dir activate_xformers link_novelai_anime_vae
!apt update
!apt install -y aria2
metalink = 'magnet:?xt=urn:btih:5bde442da86265b670a3e5ea3163afad2c6f8ecc&dn=novelaileak&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2F9.rarbg.com%3A2810%2Fannounce&tr=udp%3A%2F%2Ftracker.openbittorrent.com%3A6969%2Fannounce&tr=http%3A%2F%2Ftracker.openbittorrent.com%3A80%2Fannounce&tr=udp%3A%2F%2Fopentracker.i2p.rocks%3A6969%2Fannounce'
import re
infohash = re.search("^magnet:\?xt=urn:btih:(.*?)&.*?$", metalink).group(1)
import subprocess
tmp_dir = subprocess.check_output(['mktemp', '-d']).decode('ascii').strip('\n')
%cd "{tmp_dir}"
# Have to convert the metalink to a torrent file so aria2c can read the files inside
!aria2c -d . --bt-metadata-only=true --bt-save-metadata=true --bt-max-peers=120 --summary-interval=0 --file-allocation=none "magnet:?xt=urn:btih:5bde442da86265b670a3e5ea3163afad2c6f8ecc&dn=novelaileak&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2F9.rarbg.com%3A2810%2Fannounce&tr=udp%3A%2F%2Ftracker.openbittorrent.com%3A6969%2Fannounce&tr=http%3A%2F%2Ftracker.openbittorrent.com%3A80%2Fannounce&tr=udp%3A%2F%2Fopentracker.i2p.rocks%3A6969%2Fannounce"
!aria2c --select-file=70,71 --seed-time=0 --max-overall-upload-limit=1K --bt-max-peers=120 --summary-interval=0 --file-allocation=none "{infohash}.torrent"
!mv "novelaileak/stableckpt/animesfw-final-pruned/config.yaml" "{model_storage_dir}/novelai-animesfw-final-pruned.yaml"
!mv "novelaileak/stableckpt/animesfw-final-pruned/model.ckpt" "{model_storage_dir}/novelai-animesfw-final-pruned.ckpt"

**Hypernetworks**

A hypernetwork is trained much like a neural network and helps to guide the neural net towards the intended output.

In [ ]:
%store -r free_tier model_storage_dir repo_storage_dir activate_xformers link_novelai_anime_vae
!apt update
!apt install -y aria2
metalink = 'magnet:?xt=urn:btih:5bde442da86265b670a3e5ea3163afad2c6f8ecc&dn=novelaileak&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2F9.rarbg.com%3A2810%2Fannounce&tr=udp%3A%2F%2Ftracker.openbittorrent.com%3A6969%2Fannounce&tr=http%3A%2F%2Ftracker.openbittorrent.com%3A80%2Fannounce&tr=udp%3A%2F%2Fopentracker.i2p.rocks%3A6969%2Fannounce'
import re
infohash = re.search("^magnet:\?xt=urn:btih:(.*?)&.*?$", metalink).group(1)
import subprocess
tmp_dir = subprocess.check_output(['mktemp', '-d']).decode('ascii').strip('\n')
%cd "{tmp_dir}"
# Have to convert the metalink to a torrent file so aria2c can read the files inside
!aria2c -d . --bt-metadata-only=true --bt-save-metadata=true --bt-max-peers=120 --summary-interval=0 --file-allocation=none "magnet:?xt=urn:btih:5bde442da86265b670a3e5ea3163afad2c6f8ecc&dn=novelaileak&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2F9.rarbg.com%3A2810%2Fannounce&tr=udp%3A%2F%2Ftracker.openbittorrent.com%3A6969%2Fannounce&tr=http%3A%2F%2Ftracker.openbittorrent.com%3A80%2Fannounce&tr=udp%3A%2F%2Fopentracker.i2p.rocks%3A6969%2Fannounce"
!aria2c --select-file=76,81,82,83,84,85,86,87,88,89,90,91,92,93 --seed-time=0 --max-overall-upload-limit=1K --bt-max-peers=120 --summary-interval=0 --file-allocation=none "{infohash}.torrent"
# -exec mv doesn't work with python variables so we'll set an environment variable instead
import os
os.environ["MODEL_STORAGE_DIR"] = model_storage_dir
!rm novelaileak/stableckpt/extra-sd-prune/sd-prune/anime700k-64bs-0.1ucg-penultimate-1epoch-clip-ema-continue-76000.pt # aria2 downloads this file even though I told it not to
!find novelaileak/ -type f -name '*.pt' -exec mv {} "$MODEL_STORAGE_DIR/hypernetworks" \;

### Clean up and restart the kernel

In [ ]:
%store -r free_tier model_storage_dir repo_storage_dir activate_xformers link_novelai_anime_vae

# Get some storage back
!pip cache purge
!cd "{model_storage_dir}" && rm *.aria2
!apt remove --purge -y aria2 p7zip-full
!apt autoremove --purge -y
!apt clean

# Restart the kernel
import os
os.kill(os.getpid(), 9)

Files removed: 4
rm: cannot remove '*.aria2': No such file or directory
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package 'aria2' is not installed, so not removed
Package 'p7zip-full' is not installed, so not removed
0 upgraded, 0 newly installed, 0 to remove and 98 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 98 not upgraded.


### Link the models directory

Create symlinks. The file will be stored in the models storage directory and linked to where the WebUI expects the files to be.

In [1]:
%store -r free_tier model_storage_dir repo_storage_dir activate_xformers link_novelai_anime_vae
import os
import glob

def delete_broken_symlinks(dir):
    deleted = False
    for file in os.listdir(dir):
        path = f'{dir}/{file}'
        if os.path.islink(path) and not os.path.exists(os.readlink(path)):
            print(f'Symlink broken, removing: {file}')
            os.unlink(path)
            deleted = True
    if deleted:
        print('')
    
def symlink_models(source_dir, filetype, destination_dir):
    # don't pass directory paths with trailing slash
    for file in os.listdir(source_dir):
        if file.endswith(filetype):
            path = f'{destination_dir}/{file}'
            if not os.path.exists(path):
                print(f'New model: {file}')
                !ln -s "{source_dir}/{file}" "{destination_dir}/{file}"
            !ls -la --block-size=GB "{destination_dir}/{file}"

# Check for broken symlinks and remove them
delete_broken_symlinks(f'{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion')
delete_broken_symlinks(f'{repo_storage_dir}/stable-diffusion-webui/models/hypernetworks')

# Link models
symlink_models(model_storage_dir, 'ckpt', f'{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion')

# Link config yaml files
symlink_models(model_storage_dir, 'yaml', f'{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion')

# Link hypernetworks
symlink_models(f'{model_storage_dir}/hypernetworks', 'pt', f'{repo_storage_dir}/stable-diffusion-webui/models/hypernetworks')

# Link the NovelAI files for each of the NovelAI models
os.chdir(f'{model_storage_dir}')
for model in glob.glob('novelai-*.ckpt'):
    yaml = model.replace('.ckpt', '.yaml')
    if os.path.exists(yaml) and not os.path.exists(f'{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion/{yaml}'):        
        print(f'New NovelAI model config: {yaml}')
        !ln -s "{model_storage_dir}/{yaml}" "{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion/{yaml}"
    !ls -la --block-size=GB "{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion/{yaml}"

if link_novelai_anime_vae:
    for model in glob.glob('novelai-*.ckpt'):
        if os.path.exists(f'{model_storage_dir}/hypernetworks/animevae.pt'):
            vae = model.replace('.ckpt', '.vae.pt')
            if not os.path.exists(f'{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion/{vae}'):
                print(f'Linking NovelAI {vae} and {model}')
                !ln -s "{model_storage_dir}/hypernetworks/animevae.pt" "{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion/{vae}"
            !ls -la --block-size=GB "{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion/{vae}"
        else:
            print(f'{model_storage_dir}/hypernetworks/animevae.pt NOT FOUND')

lrwxrwxrwx 1 root root 1GB Oct 12 02:07 '/notebooks/stable-diffusion-webui/models/Stable-diffusion/ema-only-epoch=000142.ckpt' -> '/storage/models/ema-only-epoch=000142.ckpt'
lrwxrwxrwx 1 root root 1GB Oct 21 23:25 /notebooks/stable-diffusion-webui/models/Stable-diffusion/dreambooth2.ckpt -> /storage/models/dreambooth2.ckpt
lrwxrwxrwx 1 root root 1GB Oct 11 23:48 /notebooks/stable-diffusion-webui/models/Stable-diffusion/novelai-animefull-final-pruned.ckpt -> /storage/models/novelai-animefull-final-pruned.ckpt
lrwxrwxrwx 1 root root 1GB Oct 11 23:23 /notebooks/stable-diffusion-webui/models/Stable-diffusion/model.ckpt -> /storage/models/model.ckpt
lrwxrwxrwx 1 root root 1GB Oct 24 09:43 /notebooks/stable-diffusion-webui/models/Stable-diffusion/v1-5-pruned-emaonly.ckpt -> /storage/models/v1-5-pruned-emaonly.ckpt
lrwxrwxrwx 1 root root 1GB Oct 11 23:48 /notebooks/stable-diffusion-webui/models/hypernetworks/furry_protogen.pt -> /storage/models/hypernetworks/furry_protogen.pt
lrwxrwxrwx 1 ro

# Launch the WebUI

Run this block to launch the WebUI. You will get a link to nnn.gradio.app, that's your WebUI. Follow it.

- See [webui.py](https://github.com/AUTOMATIC1111/stable-diffusion-webui/blob/master/modules/shared.py#L22) to view the code for the launch args. There's a lot of good info in there about exactly what the args do. If you aren't a programmer, [here's the wiki](https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Run-with-Custom-Parameters).
- If you have a lot of VRAM and desire high generation speeds, add `--disable-opt-split-attention` to disable VRAM optimizations.
- If you have a decent about of VRAM and aren't generating large images you can remove `--medvram` for a speed boost.

#### Troubleshooting
- If you have any issues, try restarting the kernel.
- `EOFError: Ran out of input` probably means you ran out of storage space and the model `.ckpt` file wasn't downloaded completely. Try cleaning up your files. There are some helpful scripts in the Tools section below.
- If you're having issues with your results not loading, that's a known bug. I used to suggest people use ngrok but apparently accounts were getting locked because tunneling proxies are against the terms of service even though the Paperspace terms of service doesn't mention anything like that. [More details in the Tools section below.](https://github.com/anderspitman/awesome-tunneling)

In [2]:
%store -r free_tier model_storage_dir repo_storage_dir activate_xformers link_novelai_anime_vae
%cd "{repo_storage_dir}/stable-diffusion-webui"

x_arg = '--xformers' if activate_xformers else '' # enable xformers automatically

# Launch args go below:
!python webui.py {x_arg} --gradio-debug --share --medvram # --gradio-auth me:password1234

/notebooks/stable-diffusion-webui
Error loading script: wildcards.py
Traceback (most recent call last):
  File "/notebooks/stable-diffusion-webui/modules/scripts.py", line 157, in load_scripts
    compiled = compile(text, scriptfile.path, 'exec')
  File "/notebooks/stable-diffusion-webui/scripts/wildcards.py", line 1
    See the ReadMe!
        ^
SyntaxError: invalid syntax

LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
Loading weights [ab53a517] from /notebooks/stable-diffusion-webui/models/Stable-diffusion/dreambooth2.ckpt
Global Step: 5591
Applying xformers cross attention optimization.
Model loaded.
no display name and no $DISPLAY environment variable
Loaded a total of 0 textual inversion embeddings.
Embeddings: 
Running on local URL:  http://127.0.0.1:7860
Running on public 

# Export Generations

This block will rename and compress the outputs with 7zip max compression. It expects you to have `log/` and `outputs/` in `/notebooks/stable-diffusion-webui/`.

In [ ]:
%store -r free_tier model_storage_dir repo_storage_dir activate_xformers link_novelai_anime_vae
!apt update
!apt install -y p7zip-full
from datetime import datetime
datetime_str = datetime.now().strftime('%m-%d-%Y_%H:%M:%S')
%cd /notebooks/
!mkdir "{datetime_str}"
!mv "{repo_storage_dir}/stable-diffusion-webui/log" "/notebooks/{datetime_str}"
!cd "{repo_storage_dir}/stable-diffusion-webui/outputs/" && mv * "/notebooks/{datetime_str}"
!TEMP="/notebooks/{datetime_str}" # find command has issues with ipynb variables??
# !find $TEMP -name .ipynb_checkpoints -exec rm -rf "{}" +
!7z a -t7z -m0=lzma2 -mx=9 -mfb=64 -md=32m -ms=on "{datetime_str}.7z" "/notebooks/{datetime_str}/"

### Delete old output folder

This block will delete the folder you just compressed.

In [ ]:
!rm -rf "/notebooks/{datetime_str}/"
!echo Deleted /notebooks/{datetime_str}/

# Tools

### Show graphics card info

In [1]:
!nvidia-smi

Fri Oct 21 22:53:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:00:05.0 Off |                  Off |
| 30%   39C    P8    20W / 300W |      0MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Download the latest version of this notebook from Github

Run this and refresh the page (press F5). Don't save anything or you will overwrite the downloaded file.

In [ ]:
!mv /notebooks/StableDiffusionUI_Voldemort_paperspace.ipynb /notebooks/StableDiffusionUI_Voldemort_paperspace.ipynb.backup # save your old notebook to a backup
!wget https://raw.githubusercontent.com/Engineer-of-Stuff/stable-diffusion-paperspace/main/StableDiffusionUI_Voldemort_paperspace.ipynb -O /notebooks/StableDiffusionUI_Voldemort_paperspace.ipynb
!echo "Downloaded! Now, refresh the page (press F5). Don't save anything or you will overwrite the downloaded file."

### Reset Repository

Sometimes AUTOMATIC1111 breaks something. Go to https://github.com/AUTOMATIC1111/stable-diffusion-webui/commits/master and choose a commit to revert to.

If you're looking for a specific date, do: `git log --since='Sept 17 2022' --until='Sept 18 2022'`


**This shouldn't delete your outputs or any changes you've made to files, but I'd back up anything important just to be safe.**

In [ ]:
%store -r free_tier model_storage_dir repo_storage_dir activate_xformers link_novelai_anime_vae
%cd "{repo_storage_dir}/stable-diffusion-webui"
!git reset --hard <commit>

### Delete .ipynb_checkpoints

Jupyter stores temporary files in folders named `.ipynb_checkpoints`. It gets a little excessive sometimes so if you're running low on storage space or getting weird errors about a directory named `.ipynb_checkpoints`, run this block.

In [ ]:
!find . -type d -name .ipynb_checkpoints -delete

### Reset storage

This will delete ALL your files in `/notebooks/`, `/storage/`, `model_storage_dir`, and `repo_storage_dir`. Use if you're having issues with zero storage space and you don't want to delete your notebook.

In [ ]:
# Uncomment the lines below to run this block. You can highlight the lines and do ctrl + /
# %store -r free_tier model_storage_dir repo_storage_dir activate_xformers link_novelai_anime_vae
# !rm -rf /storage/*
# !mv /notebooks/*.ipynb / # move the notebook out of the directory before we nuke it
# !rm -rf /notebooks/*
# !mv /*.ipynb /notebooks/ # move it back
# !rm -rf {model_storage_dir}
# !rm -rf {repo_storage_dir}

### Download and Install Custom Scripts

This block will prompt you to enter the URL of the script you want to download.

See [https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Custom-Scripts](https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Custom-Scripts)

<mark>This is basically remote code execution so be careful.</mark>

In [ ]:
%store -r free_tier model_storage_dir repo_storage_dir activate_xformers link_novelai_anime_vae
import validators
import requests
import sys

def download_file(url, directory):
    print('Checking if text...', end='\r')
    content_type = requests.head(url).headers['content-type'].split(';')[0]
    if (content_type != 'text/plain'):
        while True:
            print(f'This is not a text file, it\'s {content_type}! Continue? y/n', end='\r')
            ask = input().lower()
            if (ask == 'y'):
                break
            if (ask == 'n'):
                sys.exit(0)
    print('\033[F' + '\033[K' + 'Downloading...', end='\r')
    local_filename = f'{directory}{url.split("/")[-1]}'
    with requests.get(url, stream=True) as r:
        try:
           r.raise_for_status()
        except requests.exceptions.HTTPError as err:
            print(err)
            raise Exception(err)
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)
    print('Downloading... done!')
    return local_filename

print('URL to script then press ENTER: ')
i = input().strip()
valid_url = validators.url(i)
if not valid_url:
    print('Not a valid URL')
else:
    download_file(i, f'{repo_storage_dir}/stable-diffusion-webui/scripts/') # this will overwrite any existing files
    print('\nScripts folder content:')
    !ls "{repo_storage_dir}/stable-diffusion-webui/scripts"

### Build and Install Xformers

This is an advanced feature that should boost your generation speeds.

1. Run the block below to download the install script to `/notebooks/`
2. Go to https://developer.nvidia.com/cuda-gpus and find the Cuda arch for your GPU model. It's likely 7.5, but double check.
3. Once you have read these instructions, uncomment the second line and insert your Cuda arch.
4. Add `--xformers` to your launch args.

If you have any issues, open the Jpyter Lab and run `build-xformers.sh` from the terminal.

In [ ]:
!wget https://raw.githubusercontent.com/Engineer-of-Stuff/stable-diffusion-paperspace/main/other/build-xformers.sh -O /notebooks/build-xformers.sh
# !bash /notebooks/build-xformers.sh [your cuda arch]
!echo "COMPLETED!"